In [67]:
import re
from datetime import datetime
import requests as rq
import pandas as pd
import numpy as np
#import lxml.etree
#import cssselect as css
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup

In [88]:
headers = {
    'User-Agent': r'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.80 Safari/537.36',
}
base_request = rq.get(r'https://wdrv.com/wdrv-music-logs/', headers=headers)
base_doc = BeautifulSoup(base_request.text)

log_pages = [(rq.post(r'https://wdrv.com/wdrv-music-logs/', headers=headers, data={'musiclogtoview': log, 'button': 'GO'}),log)
            for log in [option['value'] for option in base_doc.find(id="musiclogtoview").contents[1:]]]

In [ ]:
#todo: 
#3. export and save logs, but don't duplicate (one file per date? most recent date will not be complete)

In [89]:
def parse_page(page_pair):
    page_text = page_pair[0].text
    date_value = re.match('\d+', page_pair[1]).group(0)
    doc = BeautifulSoup(page_text)
    table = doc.find(class_='musiclog_table')
    parsed_rows = []
    for row in table('tr'):
        if not row.td.has_attr('class'):
            cells = row('td')
            time = cells[0].string
            track_match = re.fullmatch(r'(.*) by (.*)', cells[1].string)
            if track_match:
                track = track_match.group(1)
                artist = track_match.group(2)
                parsed_rows.append({
                    'track': track,
                    'artist': artist,
                    'time': datetime.strptime(date_value+time, "%Y%m%d%I:%M:%S %p")
                })
    return parsed_rows

In [98]:
parsed_frames = [pd.DataFrame(parse_page(page)) for page in log_pages]
fullframe = pd.concat(parsed_frames)

In [100]:
fullframe.groupby('track').count().sort_values(by='time', ascending=False)

,artist,time
track,,
TAKE ME HOME TONIGHT,29,29
THE BOYS ARE BACK IN TOWN,28,28
COLD AS ICE,28,28
SOMEBODY TO LOVE,27,27
CARRY ON WAYWARD SON,26,26
COME TOGETHER,26,26
TWO TICKETS TO PARADISE,25,25
RENEGADE,25,25
JUMP,25,25


In [99]:
fullframe.groupby('artist').count().sort_values(by='time', ascending=False)

,track,time
artist,,
LED ZEPPELIN,349,349
ROLLING STONES,280,280
AEROSMITH,244,244
TOM PETTY,236,236
AC/DC,211,211
VAN HALEN,205,205
BEATLES,185,185
QUEEN,151,151
PINK FLOYD,136,136
